In [7]:
import scipy.io
import torch
from thoi.measures.gaussian_copula import multi_order_measures, nplets_measures
from thoi.heuristics import simulated_annealing, greedy, simulated_annealing_multi_order
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from functools import partial
import time
device = "cuda" if torch.cuda.is_available() else "cpu"

Helper functions

In [8]:
def preprocess(array):
    reshaped_data = np.stack(array[0], axis=0)
    reshaped_data = reshaped_data.transpose(0, 2, 1)
    mean = np.mean(reshaped_data, axis=1, keepdims=True)
    std = np.std(reshaped_data, axis=1, keepdims=True)
    reshaped_normalized = (reshaped_data - mean) / std
    return reshaped_normalized

def print_time(t_i, t_f):
    elapsed_time_seconds = t_f - t_i
    hours = int(elapsed_time_seconds // 3600)
    minutes = int((elapsed_time_seconds % 3600) // 60)
    seconds = int(elapsed_time_seconds % 60)
    print("Elapsed time: {:02d}:{:02d}:{:02d}".format(hours, minutes, seconds))

Data

In [9]:
path_anesthesia = "data_ts_Anesthesia_Cleaned.mat"
data_anesthesia = scipy.io.loadmat(path_anesthesia)
ts_aw = data_anesthesia["ts_aw"]
ts_keta = data_anesthesia["ts_keta"]
ts_lpp = data_anesthesia["ts_lpp"]
ts_dpp = data_anesthesia["ts_dpp"]
ts_selv2 = data_anesthesia["ts_selv2"]
ts_selv4 = data_anesthesia["ts_selv4"]

altered_states = {
    "ketamine": preprocess(ts_keta),
    "moderate_propofol": preprocess(ts_lpp),
    "deep_propofol": preprocess(ts_dpp),
    "ts_selv2": preprocess(ts_selv2),
    "ts_selv4": preprocess(ts_selv4),
}

all_states = {
    "awake": preprocess(ts_aw),
    "ketamine": preprocess(ts_keta),
    "moderate_propofol": preprocess(ts_lpp),
    "deep_propofol": preprocess(ts_dpp),
    "ts_selv2": preprocess(ts_selv2),
    "ts_selv4": preprocess(ts_selv4),
}

print(ts_aw.shape,   "\n",
      ts_selv2.shape,"\n",
      ts_selv4.shape,"\n",
      ts_lpp.shape,  "\n",
      ts_dpp.shape,  "\n",
      ts_keta.shape)

(1, 24) 
 (1, 18) 
 (1, 11) 
 (1, 21) 
 (1, 23) 
 (1, 22)


This is the function to minimize with Simmulated Annealing. We feed it to : *simulated_annealing(metric=cohen_d)*. It takes the obtained metrics (DTC, TC, O and S) for each batch and calculates Cohen's d between awake and the selected altered state in terms of O-Information.

In [10]:
awake_subjects = all_states['awake'].shape[0]

def cohen_d(batched_measures):
    """
    In:
    - batched_measures (batch_size, D, 4) metrics: (TC, DTC, O, S)

    Returns:
    - cohen_d: (batch_size,) Cohen's d
    """
    o_Awake = batched_measures[:, :awake_subjects, 2]  # O-Information is index 2
    o_Deep = batched_measures[:, awake_subjects:, 2] 
    std_Awake = o_Awake.std(unbiased=True, axis=1)
    std_Deep = o_Deep.std(unbiased=True, axis=1)
    mean_Awake = o_Awake.mean(axis=1)
    mean_Deep = o_Deep.mean(axis=1)
    pooled_std = torch.sqrt(
        ((o_Awake.size(1) - 1) * std_Awake**2 + (o_Deep.size(1) - 1) * std_Deep**2)
        / (o_Awake.size(1) + o_Deep.size(1) - 2)
    )
    cohen_d = (mean_Deep - mean_Awake) / pooled_std
    del batched_measures, o_Awake, o_Deep, std_Awake, std_Deep, pooled_std
    torch.cuda.empty_cache()
    return cohen_d

In [ ]:

n_repeats = 2 # number of repeats to perform for Simulated Annealing, ideally closer to 100
batch_size = 100000
cohen_list = [] # to store Cohen's d values

t_i_general = time.time()

for state_name, altered_state_  in altered_states.items():
    X = np.vstack([all_states['awake'], altered_state_])
    T = [X.shape[1]] * X.shape[0]
    for order in range(3, 10): # we apply simulated annealing at each possible nplet size (or order), maximum order is 81 (there are 82 regions)
        t_i = time.time()
        print(state_name, "order", order, "batch_size:", batch_size)
        min_nplet, min_scores = simulated_annealing(
            X=X,
            order=order,
            device=device,
            T=T,
            largest=False,
            metric=cohen_d,
            repeat=n_repeats,
            batch_size=batch_size,
        )
        t_f = time.time()
        print(f"Min done, order {order}")
        print_time(t_i, t_f)
        a_ = min_nplet[min_scores.argmin().item()].detach().cpu().tolist()
        a_.sort()
        b_ = min_scores[min_scores.argmin().item()].detach().cpu().tolist()
        cohen_list.append([state_name, "min", order, a_, b_])
        torch.cuda.empty_cache()
        t_i = time.time()
        max_nplet, max_scores = simulated_annealing(
            X=X,
            order=order,
            device=device,
            T=T,
            largest=True,
            metric=cohen_d,  # Pass the custom metric
            repeat=n_repeats,
            batch_size=batch_size,
        )
        t_f = time.time()
        print(f"Max done, order: {order}")
        print_time(t_i, t_f)
        a_ = max_nplet[max_scores.argmax().item()].detach().cpu().tolist()
        a_.sort()
        b_ = max_scores[max_scores.argmax().item()].detach().cpu().tolist()
        cohen_list.append([state_name, "max", order, a_, b_])
        torch.cuda.empty_cache()
        print("Total elapsed time:")
        print_time(t_i_general, t_f)

        cols = ["state_name", "task", "order", "best_nplet", "best_score"]
        cohen_df = pd.DataFrame(cohen_list, columns=cols)
        cohen_df.to_csv(f"cohen_df_{state_name}.csv", index=False)

Example results

In [15]:
cohen_df.head(10)

,state_name,task,order,best_nplet,best_score
0,ketamine,min,3,"[22, 26, 27]",-2.085064
1,ketamine,max,3,"[29, 40, 79]",1.193535
2,ketamine,min,4,"[27, 29, 55, 69]",-2.404468
3,ketamine,max,4,"[15, 36, 67, 74]",1.213486
4,ketamine,min,5,"[26, 27, 28, 56, 81]",-3.360247
5,ketamine,max,5,"[8, 17, 21, 32, 69]",0.970320
6,ketamine,min,6,"[24, 47, 60, 67, 69, 70]",-3.478107
7,ketamine,max,6,"[36, 46, 47, 52, 70, 80]",0.841093
8,ketamine,min,7,"[5, 13, 14, 24, 42, 69, 81]",-3.261765
9,ketamine,max,7,"[1, 12, 29, 31, 36, 41, 52]",0.843628
